In [1]:
# Initialise variables and imports
import pyroomacoustics.doa as pra
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from room_simulation import Simulation
from sine_dataset import SineData
from network import SSLConvNet as ConvNet
from network import SSLConvNetCosLoss as ConvNetCosLoss
import matplotlib.pyplot as plt
import math

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
MODELS_PATH = "../models/"
DATA_PATH = "../thesis_data/"
BATCH_SIZE = 5
BATCH_SIZE_MSECOS = 25
TEST_SIZE = 10000
EPOCHS = int(TEST_SIZE/BATCH_SIZE)
EPOCHS_MSECOS = int(TEST_SIZE/BATCH_SIZE_MSECOS)
NR_MICS = 2
RAD = 50
RADII = [50, 500, 5000]
ROOM_SIMS = []
DATASETS_EXP2 = []
MIC_L_DIST = (11, -10)
MIC_R_DIST = (11, -10)
ABSORPTION = 0.0
MIN_FREQ = 20
MAX_FREQ = 20000
SAMPLE_RATE = int(MAX_FREQ*2.2)
TIME = 1
MIN_LENGTH = 65000
MIN_LENGTH_MSECOS = 48000
TO_RAD = np.pi/180
TO_DEG = 180/np.pi

# Define custom loss function
class CosBorderLoss(torch.nn.Module):

    def __init__(self):
        super(CosBorderLoss, self).__init__()

    def forward(self, pred, target):
        radial = torch.abs(torch.cos(pred-target) - torch.cos(target-target))
        border = nn.functional.relu(pred-2*np.pi) + nn.functional.relu(-pred)
        return torch.sum(radial + border)

def calcCircumPos(x1, y1, r, theta):
    xPositions = x1 + r*torch.sin(theta)
    xPositions = r*2-xPositions #Flip x-coordinates
    yPositions = y1 + r*(1-torch.cos(theta))
    yPositions = r*2-yPositions #Flip y-coordinates
    return xPositions, yPositions

def calcCircumAngle(cx, cy, x, y):
    theta = torch.atan2(cy-y, cx-x)
    theta = torch.remainder(torch.remainder(theta, 2*np.pi) + np.pi/2, 2*np.pi)
    return theta

def computeMeanITD(inL, inR):
    non_zero_ind_inL = np.argmax((inL > 0))
    non_zero_ind_inR = np.argmax((inR > 0))
    
    return (non_zero_ind_inL/SAMPLE_RATE - non_zero_ind_inR/SAMPLE_RATE) * 1000

def computeMeanIID(inL, inR):
    peak_inL_idxs = pra.detect_peaks(inL)
    peak_inR_idxs = pra.detect_peaks(inR)

    len_diff = len(peak_inL_idxs) - len(peak_inR_idxs)
    if len_diff != 0:
        if len_diff > 0:
            peak_inL_idxs = peak_inL_idxs[len_diff:]
        else:
            peak_inR_idxs = peak_inR_idxs[abs(len_diff):]
    
    return np.mean((inL[peak_inL_idxs] - inR[peak_inR_idxs]))

In [2]:
for rad in RADII:
    print(rad)
    roomSim = Simulation(SAMPLE_RATE, rad, ABSORPTION,MIC_L_DIST, MIC_R_DIST, NR_MICS)
    ROOM_SIMS.append(roomSim)

    DATASETS_EXP2.append(SineData(BATCH_SIZE, roomSim, TIME, MIN_LENGTH, MIN_FREQ, MAX_FREQ))

roomSimMSECos = ROOM_SIMS[0]
datasetMSECos = SineData(BATCH_SIZE_MSECOS, roomSimMSECos, TIME, MIN_LENGTH_MSECOS, MIN_FREQ, MAX_FREQ)

roomSim = Simulation(SAMPLE_RATE, 50, ABSORPTION,MIC_L_DIST, MIC_R_DIST, NR_MICS)
datasetLowFreq = SineData(1, roomSim, TIME, MIN_LENGTH_MSECOS, 20, 1500)
datasetMedFreq = SineData(1, roomSim, TIME, MIN_LENGTH_MSECOS, 1500, 3000)
datasetHigFreq = SineData(1, roomSim, TIME, MIN_LENGTH_MSECOS, 3000, 20000)
DATASETS_EXP3 = [datasetLowFreq, datasetMedFreq, datasetHigFreq]

50
500
5000


In [3]:
# Create networks
nets = []
for rad in RADII:
    print(rad)
    net = ConvNet(MIN_LENGTH).double()
    net.load_state_dict(torch.load(MODELS_PATH+"ConvNet_Rad"+str(rad), map_location="cpu"))
    net.eval()
    nets.append(net)

MSENet = ConvNet(MIN_LENGTH_MSECOS).double()
MSENet.load_state_dict(torch.load(MODELS_PATH+"ConvNet_MSELoss", map_location="cpu"))
MSENet.eval()

CosNet = ConvNetCosLoss(MIN_LENGTH_MSECOS).double()
CosNet.load_state_dict(torch.load(MODELS_PATH+"ConvNet_CosLoss", map_location="cpu"))
CosNet.eval()


50
500
5000


SSLConvNetCosLoss(
  (conv1): Conv1d(2, 96, kernel_size=(7,), stride=(1,), padding=(3,))
  (maxp1): MaxPool1d(kernel_size=7, stride=7, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(96, 96, kernel_size=(7,), stride=(1,), padding=(3,))
  (conv3): Conv1d(96, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (maxp3): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (maxp4): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc): Linear(in_features=35072, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=1, bias=True)
  (act): ReLU()
)

In [4]:
################
# EXPERIMENT 1 #
################

CosLoss = CosBorderLoss()
MSELoss = nn.MSELoss()
CosLosses = np.zeros((2, EPOCHS_MSECOS))
MSELosses = np.zeros((2, EPOCHS_MSECOS))
dataLoader = DataLoader(datasetMSECos, batch_size=BATCH_SIZE_MSECOS)

for i in range(EPOCHS_MSECOS):
    if i%100 == 0:
        print("Epoch "+str(i)+"/"+str(EPOCHS_MSECOS))
    
    # Retrieve all input data and labels
    inL, inR, labelX, labelY, labelAzi = next(iter(dataLoader))
    inL = inL.double().to(device)
    inR = inR.double().to(device)
    labelX = labelX.double().to(device)
    labelY = labelY.double().to(device)
    labelAzi = labelAzi.double().to(device)

    # Compute MSE and Cos losses for coordinate network
    MSENet = MSENet.to(device)
    
    outputXMSE, outputYMSE = MSENet(inL, inR)
    outputXMSE = torch.squeeze(outputXMSE)
    outputYMSE = torch.squeeze(outputYMSE)
    
    outputAziMSE = calcCircumAngle(50, 50, outputXMSE, outputYMSE)
    print("LABEL AZI = "+str(labelAzi))
    print("OUT   AZI = "+str(outputAziMSE))
    
    CosLosses[0,i] = CosLoss(outputAziMSE, labelAzi)
    MSELosses[0,i] = MSELoss(outputXMSE, labelX) + MSELoss(outputYMSE, labelY)
    
     
    # Compute MSE and Cos losses for angle network
    CosNet = CosNet.to(device)
    
    outputAziCos = CosNet(inL, inR)
    outputAziCos = torch.squeeze(outputAziCos)
    outputXCos, outputYCos = calcCircumPos(50, 0, 50, outputAziCos)
    
    CosLosses[1,i] = CosLoss(outputAziCos, labelAzi)
    MSELosses[1,i] = MSELoss(outputXCos, labelX) + MSELoss(outputYCos, labelY)

del MSENet, CosNet, inL, inR, labelX, labelY, labelAzi

np.save(DATA_PATH+"exp1_MSE_outcomes", MSELosses)
np.save(DATA_PATH+"exp1_Cos_outcomes", CosLosses)

Epoch 0/400
LABEL AZI = tensor([3.7209, 3.0073, 0.0740, 1.9320, 4.1789, 2.1447, 0.3766, 5.0191, 5.9641,
        0.8065, 5.6022, 6.0753, 1.3651, 0.0607, 6.0141, 5.0069, 4.1024, 0.3654,
        3.5709, 2.2382, 3.3005, 1.9619, 4.2048, 1.0889, 4.4510],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([3.7347, 3.0002, 0.0848, 1.9596, 4.1928, 2.1473, 0.3047, 5.0542, 5.9374,
        0.7912, 5.6240, 6.1324, 1.3726, 0.0912, 6.0220, 5.0203, 4.0846, 0.3829,
        3.5675, 2.2626, 3.3320, 1.9534, 4.2159, 1.1030, 4.4877],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([0.0382, 4.0503, 0.8643, 4.5336, 4.6558, 0.6144, 3.7040, 3.2098, 3.9255,
        2.5310, 1.0179, 5.4280, 5.0771, 3.6668, 1.8108, 1.2663, 5.1510, 4.1434,
        2.6763, 1.6400, 2.2619, 5.2673, 5.6914, 2.4935, 1.5194],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([0.0555, 4.0262, 0.8500, 4.5306, 4.6885, 0.6246, 3.6952, 3.1894, 3.9226,
        2.5534, 1.01

LABEL AZI = tensor([1.7615, 5.7850, 5.2464, 0.0366, 6.1167, 3.7029, 1.7959, 0.2065, 1.3783,
        3.2521, 0.7494, 0.3283, 5.3313, 0.3033, 0.1456, 5.7860, 0.8655, 5.3785,
        5.6703, 2.7277, 0.2043, 5.0217, 5.2460, 1.4873, 5.3907],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([1.7838, 5.7895, 5.2531, 0.0835, 6.1204, 3.7248, 1.8037, 0.2184, 1.3888,
        3.2213, 0.7639, 0.3285, 5.3558, 0.3329, 0.1695, 5.8238, 0.7898, 5.3604,
        5.6821, 2.7574, 0.2192, 5.0479, 5.2768, 1.5210, 5.3900],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([0.9030, 4.6976, 5.4941, 3.2886, 5.4841, 3.8755, 3.8217, 2.6830, 4.9574,
        2.1561, 6.0176, 1.8969, 0.0522, 4.6258, 4.9373, 1.3812, 1.6388, 4.6256,
        5.1050, 5.7269, 2.1343, 1.0072, 2.8662, 6.1900, 3.8331],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([0.9392, 4.7339, 5.5184, 3.2641, 5.5097, 3.8395, 3.8258, 2.6786, 4.9386,
        2.1710, 5.9864, 1.9000, 

LABEL AZI = tensor([2.0883e+00, 4.7758e+00, 4.7973e+00, 1.1871e-01, 4.4720e-03, 1.9969e+00,
        2.3217e+00, 2.7129e+00, 1.7249e+00, 5.7436e+00, 6.0506e+00, 2.0254e-01,
        5.1342e+00, 4.3048e+00, 1.7547e+00, 4.9115e+00, 5.6528e+00, 6.0243e+00,
        2.0031e+00, 3.7456e+00, 4.3185e+00, 4.8361e+00, 4.4053e+00, 2.1837e+00,
        3.5856e+00], device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([2.1033, 4.7917, 4.7694, 0.1499, 0.0401, 1.9868, 2.3007, 2.6965, 1.7249,
        5.7429, 6.0591, 0.2280, 5.1610, 4.2875, 1.7756, 4.9206, 5.6403, 6.0176,
        1.9782, 3.7517, 4.3175, 4.8244, 4.4137, 2.1876, 3.5917],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([0.0686, 1.5156, 3.1377, 4.3408, 3.3765, 2.3401, 2.6672, 2.1062, 1.4186,
        3.2740, 1.1829, 2.5893, 5.9235, 0.8683, 5.1019, 3.3538, 1.3865, 3.8612,
        3.4488, 2.2705, 2.4964, 3.3736, 2.8164, 4.3810, 4.4159],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = ten

LABEL AZI = tensor([1.1502, 0.7033, 0.1955, 3.7992, 3.4353, 0.4507, 4.3823, 5.0946, 0.6296,
        0.3871, 5.7234, 3.2799, 3.3673, 6.0855, 4.0381, 1.7132, 5.3600, 3.3326,
        0.5674, 2.0054, 0.1918, 4.0809, 0.2941, 1.0513, 4.0259],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([1.1720, 0.6567, 0.2017, 3.8031, 3.4443, 0.4196, 4.3969, 5.1003, 0.6071,
        0.4014, 5.7173, 3.2639, 3.3632, 6.0714, 4.0293, 1.6381, 5.4200, 3.3141,
        0.5565, 2.0211, 0.2159, 4.0664, 0.3068, 1.0582, 4.0322],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([5.4581, 0.5568, 3.2666, 2.4368, 3.4600, 3.6753, 3.9214, 2.0102, 3.3873,
        3.9762, 4.8391, 3.7230, 3.9097, 0.6503, 2.5294, 2.5579, 1.2063, 5.7790,
        0.9119, 3.6597, 3.5231, 5.0115, 5.9502, 5.6409, 2.2399],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([5.4841, 0.5552, 3.2369, 2.4348, 3.4499, 3.6213, 3.9349, 1.9968, 3.3675,
        3.9895, 4.8069, 3.7411, 

LABEL AZI = tensor([3.5372, 1.3616, 2.7375, 4.7028, 0.1996, 0.5325, 0.1355, 0.3338, 3.7288,
        2.0049, 5.9330, 2.1792, 2.3885, 3.9185, 2.9111, 3.5826, 1.0804, 4.9098,
        1.6859, 3.7513, 1.1470, 1.5289, 3.2939, 3.4407, 0.2423],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([3.5388, 1.3328, 2.7141, 4.7006, 0.2099, 0.5509, 0.0239, 0.3593, 3.7406,
        2.0333, 5.9427, 2.2171, 2.4228, 3.9133, 3.0097, 3.5863, 1.1261, 4.9011,
        1.7171, 3.7851, 1.1890, 1.4974, 3.3008, 3.4373, 0.2609],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([6.1363, 6.2812, 0.1238, 3.9685, 5.6210, 3.7128, 2.1015, 4.0005, 6.0298,
        2.8234, 5.5407, 1.9352, 2.6525, 5.3086, 2.7236, 2.4250, 5.0087, 4.1266,
        4.1178, 2.3373, 2.5584, 4.0473, 0.9675, 1.9014, 4.3916],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([6.1600, 0.0329, 0.1373, 3.9885, 5.6135, 3.7202, 2.1204, 4.0091, 6.0203,
        2.8512, 5.5424, 1.9280, 

...

LABEL AZI = tensor([3.2369, 3.9485, 1.3212, 5.6962, 2.4041, 2.1637, 4.3439, 1.4043, 5.0780,
        2.8336, 2.5831, 3.0365, 3.5993, 6.1333, 1.3038, 4.2470, 3.9730, 2.6155,
        1.0287, 3.0699, 1.8066, 4.6713, 1.6531, 2.0420, 2.1485],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([3.2177, 3.9258, 1.2839, 5.7394, 2.4333, 2.1415, 4.3299, 1.3794, 5.1018,
        2.8179, 2.5860, 3.0171, 3.5926, 6.1266, 1.3193, 4.1854, 4.0326, 2.6070,
        1.0440, 3.0342, 1.8160, 4.7125, 1.6741, 2.0592, 2.1582],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([4.6076, 4.6765, 1.9124, 2.1202, 4.0541, 5.3185, 5.3708, 4.3875, 0.3490,
        5.3429, 2.2670, 4.6645, 1.9997, 1.1906, 3.2183, 5.0078, 0.7622, 3.1488,
        3.4259, 1.0140, 0.1073, 4.3886, 2.8429, 1.7203, 5.0070],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([4.6702, 4.6723, 1.8912, 2.1211, 4.0531, 5.3168, 5.3580, 4.4378, 0.3577,
        5.3300, 2.2576, 4.6975, 

LABEL AZI = tensor([5.3873, 0.4302, 4.8802, 4.0816, 3.7322, 4.9230, 1.0591, 3.5890, 0.7384,
        2.8545, 4.4764, 1.1847, 0.1687, 3.8807, 3.9155, 1.0743, 2.6409, 0.4650,
        5.0280, 3.5157, 3.7995, 3.1446, 2.5029, 2.2752, 6.1934],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([5.3830, 0.4403, 4.9080, 4.1328, 3.7317, 4.9277, 1.0963, 3.5862, 0.7488,
        2.8396, 4.4662, 1.2229, 0.1586, 3.8984, 3.9048, 1.1053, 2.6252, 0.4700,
        5.0467, 3.5090, 3.7866, 3.1441, 2.5713, 2.2748, 6.1975],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([5.0743, 4.6399, 0.8566, 1.3977, 3.2473, 1.4531, 5.3291, 5.0376, 4.9137,
        4.8097, 3.8316, 5.3319, 5.6080, 0.5028, 5.8314, 2.1997, 1.7236, 2.4859,
        5.9560, 0.5750, 5.6350, 0.1274, 4.7734, 4.5900, 1.6303],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([5.0664, 4.6401, 0.8673, 1.3710, 3.2228, 1.4331, 5.2867, 5.0660, 4.9130,
        4.7912, 3.8285, 5.3280, 

LABEL AZI = tensor([2.5655, 0.0548, 3.1779, 3.7016, 5.4688, 4.9704, 0.0366, 1.4844, 5.1857,
        6.2541, 3.1127, 5.4545, 3.2153, 4.9300, 2.8528, 4.7315, 4.8571, 1.9623,
        1.6863, 5.6027, 1.4724, 0.8684, 0.9787, 3.0528, 5.2282],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([2.5512, 0.0777, 3.1621, 3.6846, 5.4815, 4.9861, 0.0133, 1.4752, 5.1980,
        0.0089, 3.1097, 5.4909, 3.1688, 4.9353, 2.8241, 4.7240, 4.9808, 1.9927,
        1.6693, 5.5962, 1.4592, 0.9309, 1.0139, 3.0522, 5.2273],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([2.9237, 5.3507, 1.3349, 5.2775, 2.9810, 0.1167, 4.0962, 2.5699, 0.4205,
        4.5516, 3.8443, 2.7835, 0.3043, 1.0837, 4.8537, 5.1035, 5.0605, 5.4379,
        5.7018, 4.7486, 1.4581, 4.8185, 1.4960, 0.0510, 0.4705],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([2.8891, 5.3536, 1.3439, 5.2523, 2.9607, 0.1263, 4.0728, 2.5762, 0.4258,
        4.5680, 3.8329, 2.7804, 

LABEL AZI = tensor([4.9198, 2.8902, 5.0262, 1.1721, 2.2815, 1.1297, 5.9559, 6.2281, 6.1744,
        0.5587, 4.1056, 5.7983, 4.6297, 4.6917, 4.0315, 0.0451, 3.2653, 4.1872,
        1.5379, 1.8357, 3.5628, 1.7739, 3.0031, 4.8258, 0.0631],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([4.9080, 2.8593, 5.0385, 1.2064, 2.2906, 1.1577, 5.9387, 6.2587, 6.1695,
        0.5484, 4.1062, 5.8123, 4.6330, 4.7001, 4.0452, 0.1258, 3.2414, 4.1650,
        1.4960, 1.8432, 3.5740, 1.7940, 3.0017, 4.8435, 0.0584],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([3.2487, 5.9816, 5.5472, 1.3635, 5.0994, 5.8739, 3.7623, 3.6680, 6.2537,
        2.0933, 3.6280, 0.9881, 4.1598, 2.4223, 5.2132, 3.1020, 5.7907, 1.6908,
        2.9748, 2.3107, 1.2400, 2.3026, 5.0925, 2.8893, 2.4988],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([3.2139, 5.9582, 5.5569, 1.3444, 5.1126, 5.9336, 3.7781, 3.6803, 6.2701,
        2.1370, 3.6713, 1.0273, 

LABEL AZI = tensor([0.7612, 4.4011, 0.2028, 4.8127, 2.0378, 0.6697, 5.4034, 5.8673, 1.1848,
        2.5066, 1.1657, 2.1998, 3.5709, 1.4996, 6.0717, 5.7334, 4.3233, 0.2945,
        5.1762, 1.4153, 0.4164, 3.3486, 5.1809, 5.7072, 1.7819],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([0.7817, 4.4041, 0.1966, 4.8384, 2.0805, 0.6557, 5.4058, 5.8920, 1.1808,
        2.5265, 1.1930, 2.2295, 3.5980, 1.5122, 6.1163, 5.7269, 4.3022, 0.2825,
        5.1953, 1.3962, 0.4214, 3.3450, 5.2897, 5.6959, 1.8048],
       device='cuda:1', dtype=torch.float64, grad_fn=<RemainderBackward0>)
LABEL AZI = tensor([0.2011, 5.0552, 0.3123, 3.6782, 4.1651, 3.0817, 0.6193, 0.1861, 4.8423,
        2.4188, 1.3221, 2.7569, 0.3150, 3.6486, 0.9332, 3.8945, 4.3919, 2.4497,
        1.0088, 5.7091, 5.5170, 0.0526, 5.4104, 0.7765, 4.4340],
       device='cuda:1', dtype=torch.float64)
OUT   AZI = tensor([0.2513, 5.0690, 0.3329, 3.7009, 4.1439, 3.0722, 0.6065, 0.2268, 4.9402,
        2.4285, 1.3009, 2.7665, 

In [5]:
MSELosses_exp1 = np.load(DATA_PATH+"exp1_MSE_outcomes.npy")
CosLosses_exp1 = np.load(DATA_PATH+"exp1_Cos_outcomes.npy")

print(MSELosses_exp1.shape)
print(np.mean(MSELosses_exp1, axis = 1))
print(np.std(MSELosses_exp1, axis = 1))
# print(MSELosses_exp1[:])

print(CosLosses_exp1.shape)
print(np.mean(CosLosses_exp1, axis = 1))
print(np.std(CosLosses_exp1, axis = 1))
# print(CosLosses_exp1[:])

(2, 400)
[ 5.07919129 97.51641307]
[  9.93077769 104.08698512]
(2, 400)
[0.02215426 0.48842653]
[0.07021751 0.5206958 ]


In [6]:
################
# EXPERIMENT 2 #
################

MSELosses = np.zeros((len(DATASETS_EXP2), len(nets), EPOCHS))

for i,dataset in enumerate(DATASETS_EXP2):
    print("Dataset "+str(i))

    dataLoader = DataLoader(dataset, batch_size = BATCH_SIZE)
    
    for j,net in enumerate(nets):
        print("Network "+str(j))

        net.to(device)

        for k in range(EPOCHS):
            if k%400 == 0:
                print("Epoch "+str(k)+"/"+str(EPOCHS))
            
            inL, inR, labelX, labelY, _ = next(iter(dataLoader))
            inL = inL.double().to(device)
            inR = inR.double().to(device)
            labelX = labelX.double().to(device)
            labelY = labelY.double().to(device)

            outputX, outputY = net(inL, inR)
            outputX = torch.squeeze(outputX)
            outputY = torch.squeeze(outputY)

            MSELosses[i,j,k] = MSELoss(outputX, labelX) + MSELoss(outputY, labelY)

        del net, inL, inR, labelX, labelY

np.save(DATA_PATH+"exp2_outcomes", MSELosses)

Dataset 0
Network 0
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Network 1
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Network 2
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Dataset 1
Network 0
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Network 1
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Network 2
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Dataset 2
Network 0
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Network 1
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000
Network 2
Epoch 0/2000
Epoch 400/2000
Epoch 800/2000
Epoch 1200/2000
Epoch 1600/2000


In [7]:
losses_exp2 = np.load(DATA_PATH+"exp2_outcomes.npy")

print(np.mean(losses_exp2[0], axis=1))
print(np.mean(losses_exp2[1], axis=1))
print(np.mean(losses_exp2[2], axis=1))

[4.88693906e+00 6.25207907e+05 6.91308008e+07]
[6.66632115e+05 1.11475562e+03 4.38739324e+07]
[73965010.35852115 69043399.77257815   140698.81067218]


In [9]:
################
# EXPERIMENT 3 #
################

BATCH_SIZE_EXP3 = 1
TEST_SIZE_EXP3 = 1500
EPOCHS_EXP3 = int(TEST_SIZE_EXP3/BATCH_SIZE_EXP3)
ITDs = np.zeros((len(DATASETS_EXP3), EPOCHS_EXP3))
IIDs = np.zeros((len(DATASETS_EXP3), EPOCHS_EXP3))
threshold = 2e-12
MSENet = ConvNet(MIN_LENGTH_MSECOS).double()
MSENet.load_state_dict(torch.load(MODELS_PATH+"ConvNet_MSELoss", map_location="cpu"))
MSENet.eval()
MSENet = MSENet.to(device)

for i,dataset in enumerate(DATASETS_EXP3):
    
    print("Dataset "+str(i))
    dataLoader = DataLoader(dataset, batch_size = BATCH_SIZE_EXP3)
    
    for j in range(EPOCHS_EXP3):
        
        ITDs_batch = []
        IIDs_batch = []
        
        if j%100 == 0:
                print("Epoch "+str(j)+"/"+str(EPOCHS_EXP3))
                
        inLs, inRs, _, _, _ = next(iter(dataLoader))
        
        for signalIdx in range(inLs.size(0)):
            inL = inLs[signalIdx].numpy()
            inR = inRs[signalIdx].numpy()

            inL[np.where(np.abs(inL) < threshold)] = 0
            inR[np.where(np.abs(inR) < threshold)] = 0

            ITDs_batch.append(computeMeanITD(inL, inR))
            IIDs_batch.append(computeMeanIID(inL, inR))
        
        ITDs[i,j] = np.mean(ITDs_batch)
        IIDs[i,j] = np.mean(IIDs_batch)
        
        inLs = inLs.double().to(device)
        inRs = inRs.double().to(device)
        _, _ = MSENet(inLs, inRs, record_acts = True)
        
        del inLs, inRs

import scipy.stats

for i,freq in enumerate(["LowFreq","MedFreq","HigFreq"]):
    print(freq+" correlation = " + str(scipy.stats.pearsonr(MSENet.fc_activations[1500*i:1500*(i+1)], ITDs[i])))
    print(freq+" correlation = " + str(scipy.stats.pearsonr(MSENet.fc_activations[1500*i:1500*(i+1)], IIDs[i])))

print(np.min(ITDs, axis=1))
print(np.max(ITDs, axis=1))

print(np.min(IIDs, axis=1))
print(np.max(IIDs, axis=1)) 

Dataset 0
Epoch 0/1500
Epoch 100/1500
Epoch 200/1500
Epoch 300/1500
Epoch 400/1500
Epoch 500/1500
Epoch 600/1500
Epoch 700/1500
Epoch 800/1500
Epoch 900/1500
Epoch 1000/1500
Epoch 1100/1500
Epoch 1200/1500
Epoch 1300/1500
Epoch 1400/1500
Dataset 1
Epoch 0/1500
Epoch 100/1500
Epoch 200/1500
Epoch 300/1500
Epoch 400/1500
Epoch 500/1500
Epoch 600/1500
Epoch 700/1500
Epoch 800/1500
Epoch 900/1500
Epoch 1000/1500
Epoch 1100/1500
Epoch 1200/1500
Epoch 1300/1500
Epoch 1400/1500
Dataset 2
Epoch 0/1500
Epoch 100/1500
Epoch 200/1500
Epoch 300/1500
Epoch 400/1500
Epoch 500/1500
Epoch 600/1500
Epoch 700/1500
Epoch 800/1500
Epoch 900/1500
Epoch 1000/1500
Epoch 1100/1500
Epoch 1200/1500
Epoch 1300/1500
Epoch 1400/1500
LowFreq correlation = (0.7559125598846201, 6.912154275491034e-278)
LowFreq correlation = (-0.35321250085569106, 2.616097384108685e-45)
MedFreq correlation = (0.7595867958589105, 3.834523779669846e-282)
MedFreq correlation = (-0.2963328159908376, 8.752872002568337e-32)
HigFreq correlati

In [10]:
for i,freq in enumerate(["LowFreq","MedFreq","HigFreq"]):
    print(freq+" ITD correlation = " + str(scipy.stats.pearsonr(MSENet.fc_activations[1500*i:1500*(i+1)], ITDs[i])))
    print(freq+" IID correlation = " + str(scipy.stats.pearsonr(MSENet.fc_activations[1500*i:1500*(i+1)], IIDs[i])))

print(np.min(ITDs, axis=1))
print(np.max(ITDs, axis=1))

print(np.min(IIDs, axis=1))
print(np.max(IIDs, axis=1)) 

LowFreq ITD correlation = (0.7559125598846201, 6.912154275491034e-278)
LowFreq IID correlation = (-0.35321250085569106, 2.616097384108685e-45)
MedFreq ITD correlation = (0.7595867958589105, 3.834523779669846e-282)
MedFreq IID correlation = (-0.2963328159908376, 8.752872002568337e-32)
HigFreq ITD correlation = (0.7636730841328425, 5.748070087954664e-287)
HigFreq IID correlation = (-0.25110062876319644, 5.310415690800744e-23)
[-0.65909091 -0.65909091 -0.65909091]
[0.65909091 0.65909091 0.65909091]
[-3.03803723 -3.9404468  -4.46439682]
[2.59950731 4.1069265  4.13411884]
